# Lab2 - Data Collection and Pre-processing-Jahnavi Pakanati-9013742

# Importing the required libraries 

In [112]:
import pandas as pd
from dataclasses import field
from typing import Optional
import re
from datetime import datetime
from collections import defaultdict
import json
import pyarrow as pa
import pyarrow.parquet as pq

# Loading the  raw CSV, displaying the  first 3 rows

In [113]:
df = pd.read_csv("data/clothing_transactions.csv")
print("Columns:", df.columns.tolist())
print(df.head(20))


Columns: ['date', 'customer_id', 'product', 'price', 'quantity', 'coupon_code', 'shipping_city', 'postal_code']
          date  customer_id   product   price  quantity coupon_code  \
0   2025-05-11         1000     Scarf  116.39         1     JEANS10   
1   2025-05-03         1001   T-Shirt   90.00         4     WINTER5   
2   2025-05-02         1002     Dress   80.00         2  SPRINGSALE   
3   2025-05-11         1003    Shorts   37.67         5         NaN   
4   2025-05-06         1004  Sneakers   52.64         4    SUMMER15   
5   2025-05-08         1005    Jacket  100.86         2    SUMMER15   
6   2025-05-11         1006     Dress   40.76         4    SUMMER15   
7   2025-05-06         1007     Scarf   68.01         3     CLOTH20   
8   2025-05-07         1008    Hoodie  108.53         1         NaN   
9   2025-05-08         1009     Dress   66.90         4  SPRINGSALE   
10  2025-05-04         1010    Jacket   98.25         2     WINTER5   
11  2025-05-11         1011    Short

#### Justify dict vs namedtuple vs class (1–2 sentences)
#### Dictionary--A flexible way to store data using key-value pairs.
#### NamedTuple--Like a tuple, but you can access values using names.
#### Class--A way to group data and functions together.

##### Class is the best option for me because it includes logics like cleaning, transforming and computing totals. So this will allow me to show better modularity and resuability for each transaction object.


##### Implementing the transaction class and using to populate 

In [ ]:


@dataclass
class Transaction:
    date: str
    customer_id: str
    product: str
    price: float
    quantity: int
    coupon_code: str
    shipping_city: str
    postal_code: str
    coupon_discount_rate: float = field(default=0.0)  # Deriving from the  coupon_code
    days_since_purchase: Optional[int] = field(default=None)  # Deriving  from the  purchase date

    


##### creating a method as load_transaction  and returning the list of transactions

In [115]:
def load_transactions(path: str) -> list[Transaction]:
    df = pd.read_csv(path)
    return [
        Transaction(
            date=row["date"],
            customer_id=row["customer_id"],
            product=row["product"],
            price=row["price"],
            quantity=row["quantity"],
            coupon_code=row["coupon_code"],
            shipping_city=row["shipping_city"],
            postal_code=row["postal_code"],
        ) for _, row in df.iterrows()
    ]


##### Creatiing method called profile_transactions_summary for gettting min/max/mean of price and unique city. 

In [116]:
def profile_transactions_summary(list_of_transactions) -> None:
    prices = [float(t.price) for t in list_of_transactions if isinstance(t.price, (int, float))]
    cities = {t.shipping_city for t in list_of_transactions}
    
    print("Step 5: Quick Profiling")
    print(f"Price Stats - Minimun: {min(prices)}, Mean: {sum(prices)/len(prices):.2f}, Maximum: {max(prices)}")
    print(f"Unique Shipping Cities: {len(cities)}\n")



In [117]:
list_of_transactions= load_transactions("data/clothing_transactions.csv")
profile_transactions_summary(list_of_transactions)

Step 5: Quick Profiling
Price Stats - Minimun: 15.02, Mean: 69.35, Maximum: 119.97
Unique Shipping Cities: 8



##### Injecting the dirty data 

In [118]:
def inject_dirty_values(list_of_transactions) -> None:
    if len(list_of_transactions) >= 3:
        list_of_transactions[22].price = -373435
        list_of_transactions[545].price = "adfadf"
        list_of_transactions[200].price = "unkown"
    

##### Created a function called as clean_invalid_prices and clearning values.

In [119]:
def clean_invalid_prices(list_of_transactions: list) -> None:
    before = sum(1 for t in list_of_transactions if not isinstance(t.price, float))
    
    for t in list_of_transactions:
        try:
            t.price = float(t.price)
            if t.price < 0:
                t.price = 0.0
        except:
            t.price = 0.0

    
    after = sum(1 for t in list_of_transactions if not isinstance(t.price, float))
    
    

In [120]:
list_of_transactions= load_transactions("data/clothing_transactions.csv")
inject_dirty_values(list_of_transactions)
clean_invalid_prices(list_of_transactions)

##### Making the transformations accordingly 

In [121]:
def apply_coupon_discounts(list_of_transactions: list) -> None:
    for t in list_of_transactions:
        match = re.search(r"SAVE(\d+)", str(t.coupon_code))
        t.coupon_discount_rate = int(match.group(1)) / 100 if match else 0.0


##### Doing feature engineering adding a method called days_since_purchase and trying to calculate discount 

In [122]:
def add_days_since_purchase(transactions: list) -> None:
    today = datetime.today()
    for t in transactions:
        t.days_since_purchase = (today - pd.to_datetime(t.date)).days
    

##### created a method for implementing the mini-aggregation

In [123]:
def aggregate_revenue_by_city(list_of_transactions: list) -> dict:
    city_revenue = defaultdict(float)
    for t in list_of_transactions:
        if isinstance(t.price, float):
            city_revenue [t.shipping_city] += t.price * t.quantity if t.price else 0
            
    return dict(city_revenue )

##### created a method for serialization and trying to get the json and paraquet fiels 

In [ ]:
def export_cleaned_transactions(transactions: list, json_file: str, parquet_file: str) -> None:
    data_records = [transaction.__dict__ for transaction in transactions]
    
    # Saving  as JSON
    with open(json_file, "w") as json_out:
        json.dump(data_records, json_out)
    
    # Saving as Parquet
    table = pa.Table.from_pandas(pd.DataFrame(data_records))
    pq.write_table(table, parquet_file)  
    


In [125]:
list_of_transactions = load_transactions("data/clothing_transactions.csv")
inject_dirty_values(list_of_transactions)
clean_invalid_prices(list_of_transactions)
apply_coupon_discounts(list_of_transactions)
add_days_since_purchase(list_of_transactions)
profile_transactions_summary(list_of_transactions)
city_revenue = aggregate_revenue_by_city(list_of_transactions)
export_cleaned_transactions(list_of_transactions, "data/cleaned_transactions.json", "data/cleaned_transactions.parquet")

print("City Revenue:", city_revenue)



Step 5: Quick Profiling
Price Stats - Minimun: 0.0, Mean: 69.02, Maximum: 119.97
Unique Shipping Cities: 8

City Revenue: {'Halifax': 14672.939999999997, 'Vancouver': 20080.270000000008, 'Edmonton': 17604.15, 'Montreal': 17108.41, 'Calgary': 13570.070000000002, 'Toronto': 18650.100000000006, 'Ottawa': 17658.46, 'Winnipeg': 15552.23000000001}


##### Brief explanation regarding how I used oop's 
##### Object-Oriented Programming (OOP) made it easy to divide this project into reusable, modular, and logical components. Encapsulating the # cleaning, transformation, and calculation logic into a class (`Transaction`) allowed adding behavior to each transaction record inline. This enhanced code readability and lowered the error potential since behavior and data were not separate. It also made testing and debugging a single record easier. OOP overall improved code structure and minimized the complexity of working with hundreds of records.



##### Merging the both CSV's files Clothing_transactions and address_dataset

| Field             | Type      | Description                                       | Source                |
|------------------|-----------|---------------------------------------------------|------------------------|
| date             | string    | Purchase date of the transaction                 |clothing_transactions .csv      |
| customer_id      | string    | Unique identifier for each customer              | clothing_transactions.csv      |
| product     | string    | Identifier for the product purchased             | clothing_transactions.csv      |
| postal_code     | string    | Name of the product                              | address_datasetl.csv     |
| street        | string    | Category of the street                        | address_dataset.csv     |
| city           |string    | Description of the city                       | address_dataset.csv     |
|country        | String   |Description of the country                      |address_dataset.csv
|province      | string    | Description of the province                      | address_dataset.csv     |
| price            | float     | Unit price of the product                        | clothing_transactions.csv      |
| quantity         | int       | Quantity purchased                               | clothing_transactions.csv      |
| coupon_code      | string    | Promotional code used for the transaction        | clothing_transactions.csv      |
| shipping_city    | string    | City where the product is being shipped          | clothing_transactions.csv      |
| discount_pct     | float     | Numeric discount derived from coupon_code        | Derived                |
| days_since_purchase | int   | Days since the transaction date                  | Derived                |
